# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.03039


extracting tarball to tmp_2302.03039...

 done.


Retrieving document from  https://arxiv.org/e-print/2302.03042


...

extracting tarball to tmp_2302.03425... done.
Retrieving document from  https://arxiv.org/e-print/2302.03455


extracting tarball to tmp_2302.03455...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.03576


extracting tarball to tmp_2302.03576...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03039-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03039) | **SUPER VII. Morphology and kinematics of H$α$ emission in AGN host  galaxies at Cosmic noon using SINFONI**  |
|| D. Kakkad, et al. -- incl., <mark>C. M. Harrison</mark> |
|*Appeared on*| *2023-02-08*|
|*Comments*| *16 pages, 11 figures, 2 tables, accepted for publication in MNRAS; Appendix = 2 tables and 3 figures*|
|**Abstract**| We present spatially resolved H$\alpha$ properties of 21 type 1 AGN host galaxies at z$\sim$2 derived from the SUPER survey. These targets were observed with the adaptive optics capabilities of the SINFONI spectrograph, a near-infrared integral field spectrograph, that provided a median spatial resolution of 0.3 arcsec ($\sim$2 kpc). We model the H$\alpha$ emission line profile in each pixel to investigate whether it traces gas in the narrow line region or if it is associated with star formation. To do this, we first investigate the presence of resolved H$\alpha$ emission by removing the contribution of the AGN PSF. We find extended H$\alpha$ emission in sixteen out of the 21 type 1 AGN host galaxies (76%). Based on the BPT diagnostics, optical line flux ratios and the line widths (FWHM), we show that the H$\alpha$ emission in five galaxies is ionised by the AGN (30%), in four galaxies by star formation (25%) and for the rest (45%), the ionisation source is unconstrained. Two galaxies show extended H$\alpha$ FWHM $>$600 km/s, which is interpreted as a part of an AGN-driven outflow. Morphological and kinematic maps of H$\alpha$ emission in targets with sufficient signal-to-noise ratio suggest the presence of rotationally supported disks in six galaxies and possible presence of companions in four galaxies. In two galaxies, we find an anti-correlation between the locations of extended H$\alpha$ emission and [OIII]-based ionised outflows, indicating possible negative feedback at play. However, in the majority of galaxies, we do not find evidence of outflows impacting H$\alpha$ based star formation. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03042-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03042) | **The ALMOND Survey: Molecular cloud properties and gas density tracers  across 25 nearby spiral galaxies with ALMA**  |
|| <mark>Lukas Neumann</mark>, et al. -- incl., <mark>Frank Bigiel</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Daizhong Liu</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2023-02-08*|
|*Comments*| *48 pages, 40 figures*|
|**Abstract**| We use new HCN(1-0) data from the ALMOND (ACA Large-sample Mapping Of Nearby galaxies in Dense gas) survey to trace the kpc-scale molecular gas density structure and CO(2-1) data from PHANGS-ALMA to trace the bulk molecular gas across 25 nearby, star-forming galaxies. At 2.1 kpc scale, we measure the density-sensitive HCN/CO line ratio and the SFR/HCN ratio to trace the star formation efficiency in the denser molecular medium. At 150 pc scale, we measure structural and dynamical properties of the molecular gas via CO(2-1) line emission, which is linked to the lower resolution data using an intensity-weighted averaging method. We find positive correlations (negative) of HCN/CO (SFR/HCN) with the surface density, the velocity dispersion and the internal turbulent pressure of the molecular gas. These observed correlations agree with expected trends from turbulent models of star formation, which consider a single free-fall time gravitational collapse. Our results show that the kpc-scale HCN/CO line ratio is a powerful tool to trace the 150 pc scale average density distribution of the molecular clouds. Lastly, we find systematic variations of the SFR/HCN ratio with cloud-scale molecular gas properties, which are incompatible with a universal star formation efficiency. Overall, these findings show that mean molecular gas density, molecular cloud properties and star formation are closely linked in a coherent way, and observations of density-sensitive molecular gas tracers are a useful tool to analyse these variations, linking molecular gas physics to stellar output across galaxy discs. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03576-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03576) | **JWST NIRSpec observations of Supernova 1987A -- from the inner ejecta to  the reverse shock**  |
|| J. Larsson, et al. -- incl., <mark>M. Meixner</mark>, <mark>O. Krause</mark> |
|*Appeared on*| *2023-02-08*|
|*Comments*| *Submitted to AAS Journals*|
|**Abstract**| We present initial results from JWST NIRSpec integral field unit observations of the nearby Supernova (SN) 1987A. The observations provide the first spatially-resolved spectroscopy of the ejecta and equatorial ring (ER) over the 1-5 $\mu$m range. We construct 3D emissivity maps of the [Fe I] 1.443 $\mu$m line from the inner ejecta and the He I 1.083 $\mu$m line from the reverse shock (RS), where the former probes the explosion geometry and the latter traces the structure of the circumstellar medium. We also present a model for the integrated spectrum of the ejecta. The [Fe I] 3D map reveals a highly-asymmetric morphology resembling a broken dipole, dominated by two large clumps with velocities of ~2300 km/s. We also find evidence that the Fe-rich inner ejecta have started to interact with the RS in the south. The RS surface traced by the He I line extends from just inside the ER to higher latitudes on both sides of the ER with a half-opening angle ~45 degrees, forming a bubble-like structure above and below the ER. The spectral model for the ejecta allows us to identify the many emission lines in this wavelength region, including numerous H$_2$ lines. We find that the H$_2$ is most likely excited by far-UV emission, while the metal lines ratios are consistent with recombination in very low temperature ejecta. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03425-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03425) | **Probing the nature of the low state in the extreme ultraluminous X-ray  pulsar NGC 5907 ULX1**  |
|| F. Fuerst, et al. -- incl., <mark>M. Heida</mark> |
|*Appeared on*| *2023-02-08*|
|*Comments*| *10 pages, 1 figure, accepted for publication in A&A*|
|**Abstract**| NGC 5907 ULX1 is the most luminous ultra-luminous X-ray pulsar (ULXP) known to date, reaching luminosities in excess of 1e41 erg/s. The pulsar is known for its fast spin-up during the on-state. Here, we present a long-term monitoring of the X-ray flux and the pulse period between 2003-2022. We find that the source was in an off- or low-state between mid-2017 to mid-2020. During this state, our pulse period monitoring shows that the source had spun down considerably. We interpret this spin-down as likely being due to the propeller effect, whereby accretion onto the neutron star surface is inhibited. Using state-of-the-art accretion and torque models, we use the spin-up and spin-down episodes to constrain the magnetic field. For the spin-up episode, we find solutions for magnetic field strengths of either around 1e12G or 1e13G, however, the strong spin-down during the off-state seems only to be consistent with a very high magnetic field, namely, >1e13G. This is the first time a strong spin-down is seen during a low flux state in a ULXP. Based on the assumption that the source entered the propeller regime, this gives us the best estimate so far for the magnetic field of NGC 5907 ULX1. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03455-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03455) | **Cepheid metallicity in the Leavitt law (C-MetaLL) survey -- III.  Simultaneous derivation of the Gaia parallax offset and  Period-Luminosity-Metallicity coefficients**  |
|| R. Molinaro, et al. -- incl., <mark>J. Storm</mark> |
|*Appeared on*| *2023-02-08*|
|*Comments*| *12 pages, 12 figures, accepted for publication in MNRAS*|
|**Abstract**| Classical Cepheids (DCEPs) are the most important standard candles in the extra-galactic distance scale thanks to the Period-Luminosity ($\rm PL$), Period-Luminosity-Color ($\rm PLC$) and Period-Wesenheit ($\rm PW$) relations that hold for these objects. The advent of the {\it Gaia} mission, and in particular the Early Data Release 3 (EDR3) provided accurate parallaxes to calibrate these relations. In order to fully exploit {\it Gaia} measurements, the zero point (ZP) of {\it Gaia} parallaxes should be determined with an accuracy of a few $\rm \mu as$. The individual ZP corrections provided by the {\it Gaia} team depend on the magnitude and the position on the sky of the target. In this paper, we use an implicit method that relies on the Cepheid $\rm PL$ and $\rm PW$ relations to evaluate the ensemble {\it Gaia} parallax zero point. The best inferred estimation of the offset value needed to additionally correct (after the {\it Gaia} team correction) the {\it Gaia} parallaxes of the present DCEP sample, amounts to $\rm -22\pm 4\, \mu as$. This value is in agreement with the most recent literature values and confirms that the correction proposed by the {\it Gaia} team over-corrected the parallaxes.\\ As a further application of our results, we derive an estimate of the Large Magellanic Cloud distance ($\rm \mu_0=18.49\pm 0.06\, mag$), in very good agreement with the currently accepted value obtained through geometric methods. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03105-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03105) | **Over 500 Days in the Life of the Photosphere of the Type Iax Supernova  SN 2014dt**  |
|| Yssavo Camacho-Neves, et al. -- incl., <mark>Lindsey A. Kwok</mark> |
|*Appeared on*| *2023-02-08*|
|*Comments*| *submitted to ApJ, 21 pages, 8 figures, 3 tables*|
|**Abstract**| Type Iax supernovae (SN Iax) are the largest known class of peculiar white dwarf supernovae, distinct from normal type Ia supernovae (SN Ia). The unique properties of SN Iax, especially their strong photospheric lines out to extremely late-times, allow us to model their optical spectra and derive physical parameters for the long-lasting photosphere. We present an extensive spectral timeseries, including 21 new spectra, of SN Iax 2014dt from +11 to +562 days after maximum light. We are able to reproduce the entire timeseries with a self-consistent, nearly unaltered deflagration explosion model from Fink et al. (2014) using TARDIS, an open-source radiative transfer code (Kerzendorf & Sim 2014; Kerzendorf et al. 2023). We find that the photospheric velocity of SN 2014dt slows its evolution between +64 and +148 days, which closely overlaps the phase when we see SN 2014dt diverge from the normal spectral evolution of SN Ia (+90 to +150 days). The photospheric velocity at these epochs, ~ 400--1000 km s$^{-1}$, may demarcate a boundary within the ejecta below which the physics of SN Iax and normal SN Ia differ. Our results suggest that SN 2014dt is consistent with a weak deflagration explosion model that leaves behind a bound remnant and drives an optically thick, quasi-steady-state wind creating the photospheric lines at late times. The data also suggest that this wind may weaken at epochs past +450 days, perhaps indicating a depleted radioactive power source. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03044-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03044) | **Wide-field CO isotopologue emission and the CO-to-H$_2$ factor across  the nearby spiral galaxy M101**  |
|| Jakob S. den Brok, et al. -- incl., <mark>Frank Bigiel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2023-02-08*|
|*Comments*| *Accepted for publication in A&A, 25 pages, 15 figures*|
|**Abstract**| Carbon monoxide (CO) emission is the most widely used tracer of the bulk molecular gas in the interstellar medium (ISM) in extragalactic studies. The CO-to-H$_2$ conversion factor, $\alpha_{\rm CO}$, links the observed CO emission to the total molecular gas mass. However, no single prescription perfectly describes the variation of $\alpha_{\rm CO}$ across all environments across galaxies as a function of metallicity, molecular gas opacity, line excitation, and other factors. Using resolved spectral line observations of CO and its isotopologues, we can constrain the molecular gas conditions and link them to a variation in the conversion factor. We present new IRAM 30-m 1mm and 3mm line observations of $^{12}$CO, $^{13}$CO, and C$^{18}$O} across the nearby galaxy M101. Based on the CO isotopologue line ratios, we find that selective nucleosynthesis and opacity changes are the main drivers of the variation in the line emission across the galaxy. Furthermore, we estimated $\alpha_{\rm CO(1-0)}$ using different approaches, including (i) the dust mass surface density derived from far-IR emission as an independent tracer of the total gas surface density and (ii) LTE-based measurements using the optically thin $^{13}$CO(1-0) intensity. We find an average value of $\alpha_{\rm CO}=4.4{\pm}0.9\rm\,M_\odot\,pc^{-2}(K\,km\,s^{-1})^{-1}$ across the galaxy, with a decrease by a factor of 10 toward the 2 kpc central region. In contrast, we find LTE-based values are lower by a factor of 2-3 across the disk relative to the dust-based result. Accounting for $\alpha_{\rm CO}$ variations, we found significantly reduced molecular gas depletion time by a factor 10 in the galaxy's center. In conclusion, our result suggests implications for commonly derived scaling relations, such as an underestimation of the slope of the Kennicutt Schmidt law, if $\alpha_{\rm CO}$ variations are not accounted for. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03062-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03062) | **PHANGS-MUSE: Detection and Bayesian classification of ~40000 ionised  nebulae in nearby spiral galaxies**  |
|| Enrico Congiu, et al. -- incl., <mark>Kathryn Kreckel</mark>, <mark>Frank Bigiel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2023-02-08*|
|*Comments*| *58 pages, 46 figures. Paper accepted for pubblications in A&A. The catalogue will be available via the CDS or at the following link: this http URL*|
|**Abstract**| In this work, we present a new catalogue of >40000 ionised nebulae distributed across the 19 galaxies observed by the PHANGS-MUSE survey. The nebulae have been classified using a new model-comparison-based algorithm that exploits the odds ratio principle to assign a probabilistic classification to each nebula in the sample. The resulting catalogue is the largest catalogue containing complete spectral and spatial information for a variety of ionised nebulae available so far in the literature. We developed this new algorithm to address some of the limitations of the traditional classification criteria, such as their binarity, the sharpness of the involved limits, and the limited amount of data they rely on for the classification. The analysis of the catalogue shows that the algorithm performs well when selecting H II regions. We can recover their luminosity function, and its properties are in line with what is available in the literature. We also identify a rather significant population of shock-ionised regions (mostly composed of supernova remnants), an order of magnitude larger than any other homogeneous catalogue of supernova remnants currently available in the literature. The number of supernova remnants we identify per galaxy is in line with results in our Galaxy and other very nearby sources. However, limitations in the source detection algorithm result in an incomplete sample of planetary nebulae, even though their classification seems robust. Finally, we demonstrate how applying a correction for the contribution of the diffuse ionised gas to the nebulae's spectra is essential to obtain a robust classification of the objects and how a correct measurement of the extinction using DIG-corrected line fluxes prompts the use of a higher theoretical Ha/Hb ratio (3.03) than what is commonly used when recovering the E(B-V) via the Balmer decrement technique in massive star-forming galaxies. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 1: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.03039.md
    + _build/html/tmp_2302.03039/./X_N_160_22_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./X_N_81_44_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./X_N_533_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./X_N_66_23_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./X_N_12_26_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./X_N_4_48_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./X_N_115_23_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./cid_166_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./cid_1605_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./cid_346_PSF_subtraction_K_15dec2022.png
    + _build/html/tmp_2302.03039/./X_N_115_23_HA_OIII_compare_15dec2022.png
    + _build/html/tmp_2302.03039/./cid_346_HA_OIII_compare_15dec2022.png
    + _build/html/tmp_2302.03039/./J1333_HA_OIII

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\oiii}{[\ion{O}{iii}]}$
$\newcommand{\nii}{[\ion{N}{ii}]}$
$\newcommand{\sii}{[\ion{S}{ii}]}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\oiii$}{[$\ion${O}{iii}]}$
$\newcommand{$\nii$}{[$\ion${N}{ii}]}$
$\newcommand{$\sii$}{[$\ion${S}{ii}]}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

#  emission in AGN host galaxies at Cosmic noon using SINFONI

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.03039-b31b1b.svg)](https://arxiv.org/abs/2302.03039)<mark>Appeared on: 2023-02-08</mark> - _16 pages, 11 figures, 2 tables, accepted for publication in MNRAS; Appendix = 2 tables and 3 figures_

</div>
<div id="authors">

D. Kakkad, et al. -- incl., <mark><mark>C. M. Harrison</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present spatially resolved H$\alpha$properties of 21 type 1 AGN host galaxies at z$\sim$2 derived from the SUPER survey. These targets were observed with the adaptive optics capabilities of the SINFONI spectrograph, a near-infrared integral field spectrograph, that provided a median spatial resolution of 0.3$\arcsec$($\sim$2 kpc). We model the H$\alpha$emission line profile in each pixel to investigate whether it traces gas in the narrow line region or if it is associated with star formation. To do this, we first investigate the presence of resolved H$\alpha$emission by removing the contribution of the AGN PSF. We find extended H$\alpha$emission in sixteen out of the 21 type 1 AGN host galaxies (76\%). Based on the BPT diagnostics, optical line flux ratios and the line widths (FWHM), we show that the H$\alpha$emission in five galaxies is ionised by the AGN (30\%), in four galaxies by star formation (25\%) and for the rest (45\%), the ionisation source is unconstrained. Two galaxies show extended H$\alpha$FWHM$>$600 km/s, which is interpreted as a part of an AGN-driven outflow. Morphological and kinematic maps of H$\alpha$emission in targets with sufficient signal-to-noise ratio suggest the presence of rotationally supported disks in six galaxies and possible presence of companions in four galaxies. In two galaxies, we find an anti-correlation between the locations of extended H$\alpha$emission and$\oiii$-based ionised outflows, indicating possible negative feedback at play. However, in the majority of galaxies, we do not find evidence of outflows impacting H$\alpha$based star formation.

</div>

<div id="div_fig1">

<img src="tmp_2302.03039/./X_N_160_22_PSF_subtraction_K_15dec2022.png" alt="Fig7.1" width="10%"/><img src="tmp_2302.03039/./X_N_81_44_PSF_subtraction_K_15dec2022.png" alt="Fig7.2" width="10%"/><img src="tmp_2302.03039/./X_N_533_PSF_subtraction_K_15dec2022.png" alt="Fig7.3" width="10%"/><img src="tmp_2302.03039/./X_N_66_23_PSF_subtraction_K_15dec2022.png" alt="Fig7.4" width="10%"/><img src="tmp_2302.03039/./X_N_12_26_PSF_subtraction_K_15dec2022.png" alt="Fig7.5" width="10%"/><img src="tmp_2302.03039/./X_N_4_48_PSF_subtraction_K_15dec2022.png" alt="Fig7.6" width="10%"/><img src="tmp_2302.03039/./X_N_115_23_PSF_subtraction_K_15dec2022.png" alt="Fig7.7" width="10%"/><img src="tmp_2302.03039/./cid_166_PSF_subtraction_K_15dec2022.png" alt="Fig7.8" width="10%"/><img src="tmp_2302.03039/./cid_1605_PSF_subtraction_K_15dec2022.png" alt="Fig7.9" width="10%"/><img src="tmp_2302.03039/./cid_346_PSF_subtraction_K_15dec2022.png" alt="Fig7.10" width="10%"/>

**Figure 7. -** The plots in this figure show the results of the PSF-subtraction procedure described in Sect. \ref{sect3.2}. The maps show the PSF-subtracted H$\alpha$ channel map and the corresponding right panels show the spectrum extracted from regions above 3$\sigma$ noise levels (white regions in the channel maps). The noise levels in each map is obtained from object-free regions. The "X" in the maps mark the peak of the K-band continuum emission which is used as a proxy for the AGN location. The magenta shaded region in the spectra show the channels that were collapsed to obtain the PSF-subtracted images. The vertical red dashed line shows the expected location of the H$\alpha$ line based on the redshift of the respective targets. The presence of structure in the PSF-subtracted map and a visible detection of the emission line in the spectra would suggest an extended H$\alpha$ feature. Each target in this figure show signatures of extended H$\alpha$ emission (labelled as "Resolved"), except X\_N\_66\_23 and cid\_166 (labelled as "Resolved?") where the extension is a possibility but unconfirmed with the current data. Further details are given in Sect. \ref{sect3.2}. (*fig:BPT_PSF*)

</div>
<div id="div_fig2">

<img src="tmp_2302.03039/./X_N_115_23_HA_OIII_compare_15dec2022.png" alt="Fig2.1" width="20%"/><img src="tmp_2302.03039/./cid_346_HA_OIII_compare_15dec2022.png" alt="Fig2.2" width="20%"/><img src="tmp_2302.03039/./J1333_HA_OIII_compare_15dec2022.png" alt="Fig2.3" width="20%"/><img src="tmp_2302.03039/./S82X1905_HA_OIII_compare_15dec2022.png" alt="Fig2.4" width="20%"/><img src="tmp_2302.03039/./BPT.png" alt="Fig2.5" width="20%"/>

**Figure 2. -** The background images in the maps in the top two rows shows the PSF-subtracted H$\alpha$ images and the overlaid blue contours show the locations of extended $\oiii$ emission. The contour levels are at 98\%, 90\%, 70\% and 50\% of the peak $\oiii$ flux. In four targets, X\_N\_115\_23, cid\_346, J1333+1649 and S82X1905, the spatial location of the extended H$\alpha$ emission coincides with that of the $\oiii$ emission. This made it possible to place the extended H$\alpha$ emission of these four galaxies in the BPT diagram shown in the bottom panel. In all these four galaxies, the extended emission is ionised by the AGN. The black line shows the division between star forming and AGN ionisation from \citet{kewley13}. Targets, X\_N\_81\_44 and J1549+1245 do not show the same location of the $\oiii$ and H$\alpha$ emission. (*fig:BPT*)

</div>
<div id="div_fig3">

<img src="tmp_2302.03039/./cid_1205_PSF_subtraction_K_15dec2022.png" alt="Fig8.1" width="14%"/><img src="tmp_2302.03039/./cid_467_PSF_subtraction_K_15dec2022.png" alt="Fig8.2" width="14%"/><img src="tmp_2302.03039/./J1333_PSF_subtraction_K_15dec2022.png" alt="Fig8.3" width="14%"/><img src="tmp_2302.03039/./J1549_PSF_subtraction_K_15dec2022.png" alt="Fig8.4" width="14%"/><img src="tmp_2302.03039/./S82X1905_PSF_subtraction_K_15dec2022.png" alt="Fig8.5" width="14%"/><img src="tmp_2302.03039/./S82X1940_PSF_subtraction_K_15dec2022.png" alt="Fig8.6" width="14%"/><img src="tmp_2302.03039/./S82X2058_PSF_subtraction_K_15dec2022.png" alt="Fig8.7" width="14%"/>

**Figure 8. -** Same as Fig. \ref{fig:BPT_PSF}, for the rest of the targets presented in this paper. All the targets, except cid\_1205 and J1549+1245, show confirmed evidence of extended H$\alpha$ emission. We detected no extended emission in cid\_1205 and the detection in J1549+1245 is tentative. (*fig:BPT_PSF2*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\cmark}{\textcolor{green}{\ding{51}}}$
$\newcommand{\xmark}{\textcolor{red}{\ding{55}}}$
$\newcommand{\ubonn}{Argelander-Institut f\"ur Astronomie, Universit\"at Bonn, Auf dem H\"ugel 71, 53121 Bonn, Germany}$
$\newcommand{\osu}{Department of Astronomy, The Ohio State University, 4055 McPherson Laboratory, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\newcommand{\oan}{Observatorio Astronómico Nacional (IGN), C/ Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\inaf}{INAF — Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Florence, Italy}$
$\newcommand{\mpe}{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\newcommand{\zah}{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur theoretische Astrophysik, Albert-Ueberle-Stra{\ss}e 2, 69120, Heidelberg, Germany}$
$\newcommand{\cool}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\anu}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\astrothreed}{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\newcommand{\mpia}{Max Planck Institute for Astronomy, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{\hsc}{Centro de Desarrollos Tecnol\'ogicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain}$
$\newcommand{\zw}{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, 69120 Heidelberg, Germany}$
$\newcommand{\gent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\iram}{Institut de Radioastronomie Millim\'etrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d’Hères, France}$
$\newcommand{\lerma}{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris, France}$
$\newcommand{\ucsd}{Center for Astrophysics and Space Sciences, Department of Physics, University of California San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\mmu}{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\cita}{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\wyo}{Department of Physics \& Astronomy, University of Wyoming, Laramie, WY, 82071, USA}$
$\newcommand{\alb}{Dept. of Physics, University of Alberta, Edmonton, Alberta, Canada T6G 2E1}$
$\newcommand{\ox}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\ESO}{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei M\"{u}nchen, Germany}$
$\newcommand{\ljmu}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{\SAO}{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden St., 02138 Cambridge, MA, USA}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\SIUnitSymbolDegree}{\textsuperscript{h}}$
$\newcommand{\SIUnitSymbolArcminute}{\textsuperscript{m}}$
$\newcommand{\SIUnitSymbolArcsecond}{\textsuperscript{s}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\cmark$}{\textcolor{green}{\ding{51}}}$
$\newcommand{$\xmark$}{\textcolor{red}{\ding{55}}}$
$\newcommand{$\ubonn$}{Argelander-Institut f\"ur Astronomie, Universit\"at Bonn, Auf dem H\"ugel 71, 53121 Bonn, Germany}$
$\newcommand{$\osu$}{Department of Astronomy, The Ohio State University, 4055 McPherson Laboratory, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\newcommand{$\oan$}{Observatorio Astronómico Nacional (IGN), C/ Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{$\inaf$}{INAF — Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Florence, Italy}$
$\newcommand{$\mpe$}{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\newcommand{$\zah$}{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur theoretische Astrophysik, Albert-Ueberle-Stra{\ss}e 2, 69120, Heidelberg, Germany}$
$\newcommand{$\cool$}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{$\anu$}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{$\astrothreed$}{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\newcommand{$\mpia$}{Max Planck Institute for Astronomy, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{$\hsc$}{Centro de Desarrollos Tecnol\'ogicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain}$
$\newcommand{$\zw$}{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, 69120 Heidelberg, Germany}$
$\newcommand{$\gent$}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{$\iram$}{Institut de Radioastronomie Millim\'etrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d’Hères, France}$
$\newcommand{$\lerma$}{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris, France}$
$\newcommand{$\ucsd$}{Center for Astrophysics and Space Sciences, Department of Physics, University of California San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{$\mmu$}{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{$\cita$}{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\newcommand{$\wyo$}{Department of Physics \& Astronomy, University of Wyoming, Laramie, WY, 82071, USA}$
$\newcommand{$\alb$}{Dept. of Physics, University of Alberta, Edmonton, Alberta, Canada T6G 2E1}$
$\newcommand{$\ox$}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{$\ESO$}{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei M\"{u}nchen, Germany}$
$\newcommand{$\ljmu$}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{$\SAO$}{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden St., 02138 Cambridge, MA, USA}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{$\SIUnitSymbolDegree$}{\textsuperscript{h}}$
$\newcommand{$\SIUnitSymbolArcminute$}{\textsuperscript{m}}$
$\newcommand{$\SIUnitSymbolArcsecond$}{\textsuperscript{s}}$</div>



<div id="title">

# The ALMOND Survey: Molecular cloud properties and gas density tracers across 25 nearby spiral galaxies with ALMA

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.03042-b31b1b.svg)](https://arxiv.org/abs/2302.03042)<mark>Appeared on: 2023-02-08</mark> - _48 pages, 40 figures_

</div>
<div id="authors">

<mark><mark>Lukas Neumann</mark></mark>, et al. -- incl., <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Jonathan D. Henshaw</mark></mark>, <mark><mark>Daizhong Liu</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** We use new\hcnonedata from the ALMOND (ACA Large-sample Mapping Of Nearby galaxies in Dense gas) survey to trace the kpc-scale molecular gas density structure and\cotwodata from PHANGS-ALMA to trace the bulk molecular gas across 25 nearby, star-forming galaxies.At\SI{2.1}{\kilo\parsec}scale, we measure the density-sensitive HCN/CO line ratio and the SFR/HCN ratio to trace the star formation efficiency in the denser molecular medium.At\SI{150}{\parsec}scale, we measure structural and dynamical properties of the molecular gas via\cotwoline emission, which is linked to the lower resolution data using an intensity-weighted averaging method.We find positive correlations (negative) of HCN/CO (SFR/HCN) with the surface density, the velocity dispersion and the internal turbulent pressure of the molecular gas.These observed correlations agree with expected trends from turbulent models of star formation, which consider a single free-fall time gravitational collapse.Our results show that the kpc-scale HCN/CO line ratio is a powerful tool to trace the$\SI{150}{\parsec}$scale average density distribution of the molecular clouds.Lastly, we find systematic variations of the SFR/HCN ratio with cloud-scale molecular gas properties, which are incompatible with a universal star formation efficiency.Overall, these findings show that mean molecular gas density, molecular cloud properties and star formation are closely linked in a coherent way, and observations of density-sensitive molecular gas tracers are a useful tool to analyse these variations, linking molecular gas physics to stellar output across galaxy discs.

</div>



<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\msun}{ M_\odot  }$
$\newcommand{\dg}{^\circ}$
$\newcommand{\kms}{\rm{km s^{-1}}}$
$\newcommand{\ergcms}{\rm erg~cm^{-2}~s^{-1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\msun$}{ M_\odot  }$
$\newcommand{$\dg$}{^\circ}$
$\newcommand{$\kms$}{\rm{km s^{-1}}}$
$\newcommand{$\ergcms$}{\rm erg~cm^{-2}~s^{-1}}$</div>



<div id="title">

# JWST NIRSpec observations of Supernova 1987A -- from the inner ejecta to the reverse shock 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.03576-b31b1b.svg)](https://arxiv.org/abs/2302.03576)<mark>Appeared on: 2023-02-08</mark> - _Submitted to AAS Journals_

</div>
<div id="authors">

J. Larsson, et al. -- incl., <mark>M. Meixner</mark>, <mark>O. Krause</mark>

</div>
<div id="abstract">

**Abstract:** We present initial results from{\it JWST}NIRSpec integral field unit observations of the nearby Supernova (SN) 1987A. The observations provide the first spatially-resolved spectroscopy of the ejecta and equatorial ring (ER) over the 1--5$\mu$m range. We construct 3D emissivity maps of the[$\ion${Fe}{1}]$1.443 \mu$m line from the inner ejecta and the$\ion${He}{1}1.083$\mu$m line from the reverse shock (RS), where the former probes the explosion geometry and the latter traces the structure of the circumstellar medium. We also present a model for the integrated spectrum of the ejecta. The[$\ion${Fe}{1}]3D map reveals a highly-asymmetric morphology resembling a broken dipole, dominated by two large clumps with  velocities of$\sim 2300$\kms. We also find evidence that the Fe-rich inner ejecta have started to interact with the RS in the south. The RS surface traced by the$\ion${He}{1}line extends from just inside the ER to higher latitudes on both sides of the ER with a half-opening angle$\sim 45$\dg, forming a bubble-like structure above and below the ER. The spectral model for the ejecta allows us to identify the many emission lines in this wavelength region, including numerous H$_2$lines. We find that the H$_2$is most likely excited by far-UV emission, while the metal lines ratios are consistent with recombination in very low temperature ejecta.

</div>

<div id="div_fig1">

<img src="tmp_2302.03576/./fe1p44_slices_500kms_5x4_er.png" alt="Fig4" width="100%"/>

**Figure 4. -** Images of the [$\ion${Fe}{1}] 1.443 $\mu$m emission from the ejecta as a function of Doppler shift, ranging from $V\rm_z = -4000$ $\kms$(top left) to $V\rm_z =5000$ $\kms$(bottom right). Each image was integrated over an interval of 500 $\kms$. The velocities of freely-expanding ejecta in the plane of the sky are shown on the x and y axes, where the assumed center of explosion marks 0 velocity (white star symbol). The dashed white line shows the position of the ER. (*fig:fe_slices*)

</div>
<div id="div_fig2">

<img src="tmp_2302.03576/./specregions_fe1p44_image_fullfov_lab.png" alt="Fig1" width="100%"/>

**Figure 1. -** NIRSpec image integrated over the wavelength region of [$\ion${Fe}{1}] 1.443 $\mu \rm{m} \pm 5000$ $\kms$. This line originates from the ejecta, while the emission from the ER in this wavelength range is due to narrow [$\ion${Fe}{1}] $1.427 \mu$m and $\ion${H}{1} $1.460 \mu$m lines, as well as continuum. The white lines show the regions used to extract spectra for the ejecta (inner ellipse) and ER (elliptical annulus). The five point sources seen outside the ER are stars. (*fig:fe_image*)

</div>
<div id="div_fig3">

<img src="tmp_2302.03576/./fe1p44_3dvol_3views.png" alt="Fig5" width="100%"/>

**Figure 5. -** Volume rendering of the [$\ion${Fe}{1}] 1.443 $\mu$m emission from the ejecta. The three panels show the system from different viewing angles, as indicated by the arrows in the lower left corners. In the left and middle panels, the faintest emission (plotted in blue) corresponds to 15\% of the peak value (red). The right panel only shows ejecta with $V\rm_z >1000$ $\kms$ in order to highlight the faint ring structure. In this case, the faintest emission plotted corresponds to 10\% of the peak value. The gray circle shows the inner edge of the ER. (*fig:fe_3d*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

195  publications in the last 7 days.
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers